In [2]:
#saccades
#three or five digits
#transparsenecy 

In [7]:
%cd /Users/typhaine/Documents/Doc_Gorilla/WhereIsMyMNIST/notebooks

/Users/typhaine/Documents/Doc_Gorilla/WhereIsMyMNIST/notebooks


In [8]:
import os
import imageio
import random
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import pygame
from psychopy import visual, core, event, gui, data

from what import WhatShift, WhatBackground, WhatNet, WhatTrainer, What, train, test, MNIST, MotionCloudNoise
import MotionClouds as mc

from main import init
args = init(filename='../data/2019-06-12') # pas de drop out!
args


{'w': 28,
 'minibatch_size': 100,
 'train_batch_size': 50000,
 'test_batch_size': 10000,
 'noise_batch_size': 1000,
 'mean': 0.1307,
 'std': 0.3081,
 'what_offset_std': 15,
 'what_offset_max': 25,
 'N_pic': 128,
 'offset_std': 30,
 'offset_max': 34,
 'noise': 0.75,
 'contrast': 0.7,
 'sf_0': 0.1,
 'B_sf': 0.1,
 'do_mask': True,
 'N_theta': 6,
 'N_azimuth': 24,
 'N_eccentricity': 10,
 'N_phase': 2,
 'rho': 1.41,
 'bias_deconv': True,
 'p_dropout': 0.0,
 'dim1': 1000,
 'dim2': 1000,
 'lr': 0.005,
 'do_adam': True,
 'bn1_bn_momentum': 0.5,
 'bn2_bn_momentum': 0.5,
 'momentum': 0.3,
 'epochs': 60,
 'num_processes': 1,
 'no_cuda': False,
 'log_interval': 100,
 'verbose': 1,
 'filename': '../data/2019-06-12',
 'seed': 2019,
 'N_cv': 10,
 'do_compute': True,
 'save_model': True}

In [9]:
## Unit tests

#transforms.GaussianBlur(kernel_size = 1, sigma=(0.1, 2.0))
transform=transforms.Compose([
                               WhatShift(args,i_offset=0, j_offset=0),
                               WhatBackground(contrast = args.contrast,
                                              noise=0, 
                                              sf_0=args.sf_0, 
                                              B_sf=args.B_sf),
                               transforms.ToTensor(),
                               transforms.Normalize((args.mean,), (args.std,))
                           ])

dataset_train = MNIST('../data',
                        train=True,
                        download=True,
                        transform=transform,
                        )

train_loader = torch.utils.data.DataLoader(dataset_train,
                                         batch_size=args.minibatch_size,
                                         shuffle=True)

dataset_test = MNIST('../data',
                        train=False,
                        download=True,
                        transform=transform,
                        )

test_loader = torch.utils.data.DataLoader(dataset_test,
                                         batch_size=args.minibatch_size,
                                         shuffle=True)

data, label = next(iter(train_loader))
#%timeit data, label = next(iter(train_loader))
print(label.shape)
print(data.shape)

torch.Size([100])
torch.Size([100, 1, 128, 128])


In [10]:
def add(data):
    data = data.numpy().reshape(100,128,128)
    return np.array([numbers(data) for i in range(100)]).reshape(100,128,128)

In [11]:
def numbers(data):
    num = [
    random_roll(data[random.randint(0,99)],random.randint(30,50),random.randint(30,50)),
    random_roll(data[random.randint(0,99)],random.randint(-50,-30),random.randint(-50,-30)),
    random_roll(data[random.randint(0,99)],random.randint(-50,-30),random.randint(30,50)),
    random_roll(data[random.randint(0,99)],random.randint(30,50),random.randint(-50,-30)),
    random_roll(data[random.randint(0,99)],random.randint(-50,-30),random.randint(0,20)),
    random_roll(data[random.randint(0,99)],random.randint(0,20),random.randint(-50,-20)),
    random_roll(data[random.randint(0,99)],random.randint(30,50),random.randint(0,20)),
    random_roll(data[random.randint(0,99)],random.randint(0,20),random.randint(30,50)),
    random_roll(data[random.randint(0,99)],random.randint(0,20),random.randint(0,20))]
    np.random.shuffle(num)
    return np.max((num[0:random.randint(1,9)]),axis = 0)

In [12]:
def RGBA(data):
    data = np.interp(data, (data.min(), data.max()), (0, 1))
    alpha = np.ones((data.shape[0],128,128))
    return np.stack((data,data,data,alpha),axis=-1)

In [13]:
def random_roll(data, x_shift, y_shift):
    return np.roll(np.roll(data, x_shift, axis = 0), y_shift, axis = 1)

In [14]:
def motion(data,background):
    data = np.interp(data, (data.min(), data.max()), (0, 1))
    background = np.reshape(RGBA(background),(128,128,4))
    return np.swapaxes(
        np.moveaxis(
            np.array([[
                np.max((
                    np.roll(background,back,axis =(0,1)),data[D,:,:,:]),axis=0) 
                for back in range(60)] 
                for D in range(data.shape[0])]),(0,1,2,3,4),(0,1,4,3,2)),3,4)

# np.swapaxes(x,3,4)
#np.moveaxis(x,(0,1,2,3,4),(0,1,4,3,2))
            

In [15]:
def transparency(data):
    [[[np.put(img[frame,3,row,:],np.where(img[frame,0,row,:]>0.9),np.linspace(0, 1, num=60)[frame]) 
       for row in range(128)] 
      for frame in range(60)] for img in data]



In [16]:
def transparency(data):
    for img in data:
        for frame in range(60):
            t = np.linspace(0, 1, num=60)[frame]
            for row in range(128):
                p = np.where(img[frame,0,0,:]>0.9)
                img[frame,3,,:]
                        
                
            

SyntaxError: invalid syntax (<ipython-input-16-a9a6206bbf13>, line 7)

In [17]:
#%timeit z, env = MotionCloudNoise(sf_0=args.sf_0, B_sf=args.B_sf, alpha=.0, N_pic=128, seed=42, motion = True)
z, env = MotionCloudNoise(sf_0=args.sf_0, B_sf=args.B_sf, alpha=.0, N_pic=128, seed=42, motion = True)
#%timeit motion(RGBA(add(data)),np.moveaxis(z,(0,1,2),(2,1,0)))
data = motion(RGBA(add(data)),np.moveaxis(z,(0,1,2),(2,1,0)))
#%timeit transparency(data)
#transparency(data)


In [19]:
T= numba.njit()(transparency)

In [20]:
%time T(data)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mUse of unsupported NumPy function 'numpy.put' or unsupported use of the function.
[1m
File "<ipython-input-12-a43b14301167>", line 3:[0m
[1mdef transparency(data):
    <source elided>
    [[[np.put(img[frame,3,row,:],np.where(img[frame,0,row,:]>0.9),np.linspace(0, 1, num=60)[frame]) 
[1m       for row in range(128)] 
[0m       [1m^[0m[0m
[0m
[0m[1m[1] During: typing of get attribute at <ipython-input-12-a43b14301167> (3)[0m
[1m
File "<ipython-input-12-a43b14301167>", line 3:[0m
[1mdef transparency(data):
    <source elided>
    [[[np.put(img[frame,3,row,:],np.where(img[frame,0,row,:]>0.9),np.linspace(0, 1, num=60)[frame]) 
[1m       for row in range(128)] 
[0m       [1m^[0m[0m

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba vi

In [28]:
data.shape

(100, 60, 4, 128, 128)

In [27]:
instructions = """
At each trial you see a series of textured images, report if you 
spot a specific digit by pressing the space bar.

Press the space bar to begin the experiment.

"""

win = visual.Window(winType = 'pyglet', size =(800,800), colorSpace = "rgb")

message = visual.TextStim(win, text= instructions)
message.autoDraw = True  # Automatically draw every frame
win.flip()
core.wait(5.0)
message.text = ''  # Change properties of existing stim
win.flip()
core.wait(0.0)

while True:
    for i in range(data.shape[0]-1):
            mov = data[i]
            for ii in range(mov.shape[0]):
                movie_clip = visual.ImageStim(win, np.moveaxis(mov[ii],(0,1,2),(2,1,0)), colorSpace = "rgb", contrast = 1, size = 2, flipVert=True)
                movie_clip.draw()
                win.flip()

KeyboardInterrupt: 

In [ ]:
def show_landmarks(image, landmarks):
    """Show image with landmarks"""
    plt.imshow(image)
    plt.scatter(landmarks[:, 0], landmarks[:, 1], s=100, marker='.', c='r')
        # s is just the siz of the marker
        # c is just the colour
            # Reference: https://matplotlib.org/3.3.3/api/_as_gen/matplotlib.pyplot.scatter.html
        # landmarks[:, 0], landmarks[:, 1]
            # Numpy/Pandas notation, means more or less
                # Reference: https://stackoverflow.com/questions/40557910/plt-plot-meaning-of-0-and-1/40558730
    # plt.pause(0.001)  # pause a bit so that plots are updated

for key in loc_data_xy: # loc_data_xy is a dictionary so there is a key:value
    # print(x,loc_data_xy[x])
    plt.figure()
    show_landmarks(io.imread(os.path.join('/Users/typhaine/Documents/Doc Gorilla/ALLSTIMULI/' + key +".jpeg")),
               loc_data_xy[key])
    # loc_data_xy[x] accesses values
    plt.show()
    if key == key:
        break
        # Sloppy, but hey it works!
        
# Reference for loop: https://stackoverflow.com/questions/26660654/how-do-i-print-the-key-value-pairs-of-a-dictionary-in-python/26660785